In [ ]:
#Importing libraries
import os
import numpy
import matplotlib.pyplot as plt
import pandas
import pickle
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

#Importing prompt and afterglow indexes using pickle
with open('StartandFinish.pickle','rb') as f:
    prompt_start, prompt_finish, afterglow_start, afterglow_finish = pickle.load(f)

#Setting the directory
directory = os.getcwd()
desktop_dir = 'C:\\Users\\Alb\\Desktop'
if directory == desktop_dir:
    os.chdir('Raw Data')
    
#Listing files and setting column names
files_list = os.listdir()
col_names = ['Time', 'Time_err_p', 'Time_err_n', 'Fluence', 'Fluence_err_p', 'Fluence_err_n']

#Fonksiyon tanımı
def create_dataset(dataset,look_back=1):
    dataX, dataY = [],[]
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back),0]
        dataX.append(a)
        dataY.append(dataset[i+look_back,0])
    return numpy.array(dataX), numpy.array(dataY)

i=21 # For sGRB 051221A
#Creating the dataframe
dataframe = pandas.read_csv(files_list[i], sep='\t', names=col_names, engine='python')
dataframe = dataframe.drop(columns=['Time','Time_err_p', 'Time_err_n', 'Fluence_err_p', 'Fluence_err_n'])
dataset = dataframe.values
dataset = dataset.astype('float32')

#Extracting only the prompt and afterglow fluences in a single column
prompt = dataset[prompt_start[i]:prompt_finish[i]]
afterglow = dataset[afterglow_start[i]:afterglow_finish[i]]
dataset = numpy.concatenate((prompt,afterglow))


#Veri setinin normalize edilmesi
scaler = MinMaxScaler(feature_range=(0,1))
dataset = scaler.fit_transform(dataset)

#Train test split
train_size = int(len(dataset)*0.2)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train),len(test))
    

#Matrislerin yeniden şekillendirilmesi
look_back=1
trainX, trainY = create_dataset(train,look_back)
testX, testY = create_dataset(test, look_back)

#Matrisi LSTM tarzına dönüştürme [sample, time step, feature]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

#LSTM modelinin hazırlanması
model = Sequential()
model.add(LSTM(4, input_shape=(1,look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs=200,batch_size=1, verbose=2)

#Tahminler ve hata hesabı
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)
# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))


# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset), label='Veri', color='black')
plt.plot(trainPredictPlot, label='Train tahmini', color='blue')
plt.plot(testPredictPlot, label='Test tahmini', color='red')
plt.title('GRB 050502B, LSTM Tahmini')
plt.xlabel('Veri noktası')
plt.ylabel('Fluence (erg/cm^2/2)')
plt.legend(loc='upper right')
plt.show()